In [70]:
import pandas as pd
import json
import numpy as np
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy import stats

with open('cleaned_entities.json', 'r') as file:
    data = json.load(file)

# The rest of the code remains the same
transformed_data = []
for server, details in data.items():
    details['server'] = server
    transformed_data.append(details)
dataList1 = transformed_data[0]
dataList2 = transformed_data[1]

character_list = []

for name, attributes in dataList1.items():
    # print(f'{name}\n{attributes}\n{type(attributes)}')
    if not isinstance(attributes,str):
        character_attributes = attributes.copy()  # Create a copy of the character attributes
        character_attributes["name"] = name  # Add the "name" key with the character's name
        character_list.append(character_attributes)  # Append the updated attributes to the list

df1 = pd.DataFrame(character_list)

character_list = []

for name, attributes in dataList2.items():
    # print(f'{name}\n{attributes}\n{type(attributes)}')
    if not isinstance(attributes,str):
        character_attributes = attributes.copy()  # Create a copy of the character attributes
        character_attributes["name"] = name  # Add the "name" key with the character's name
        character_list.append(character_attributes)  # Append the updated attributes to the list

df2 = pd.DataFrame(character_list)

df1 = df1.drop(['guild','date','map_pos','source_id','name','last_words','in_guild','instance_id'],axis=1)
df2 = df2.drop(['guild','date','map_pos','source_id','name','last_words'],axis=1)

df = pd.concat([df1,df2]).sort_values(by='level', ascending=True).dropna().reset_index(drop=True).reindex(columns=['level','class_id','race_id','map_id']).astype(int)


In [71]:
### Codes from Deathlog

classCodeDict = {
    0: "None Selected",
    1: "Warrior",
    2: "Paladin",
    3: "Hunter",
    4: "Rogue",
    5: "Priest",
    7: "Shaman",
    8: "Mage",
    9: "Warlock",
    11: "Druid"
}

environmentDamageCodeDict = {
    -2: "Drowning",
    -3: "Falling",
    -4: "Fatigue",
    -5: "Fire",
    -6: "Lava",
    -7: "Slime"
}

raceCodeDict = {
    0: "None Selected",
    1: "Human",
    2: "Orc",
    3: "Dwarf",
    4: "Night Elf",
    5: "Undead",
    6: "Tauren",
    7: "Gnome",
    8: "Troll"
}


zoneCodeDict = {
    947: "Azeroth",
    1411: "Durotar",
    1412: "Mulgore",
    1413: "The Barrens",
    1414: "Kalimdor",
    1415: "Eastern Kingdoms",
    1416: "Alterac Mountains",
    1417: "Arathi Highlands",
    1418: "Badlands",
    1419: "Blasted Lands",
    1420: "Tirisfal Glades",
    1421: "Silverpine Forest",
    1422: "Western Plaguelands",
    1423: "Eastern Plaguelands",
    1424: "Hillsbrad Foothills",
    1425: "The Hinterlands",
    1426: "Dun Morogh",
    1427: "Searing Gorge",
    1428: "Burning Steppes",
    1429: "Elwynn Forest",
    1430: "Deadwind Pass",
    1431: "Duskwood",
    1432: "Loch Modan",
    1433: "Redridge Mountains",
    1434: "Stranglethorn Vale",
    1435: "Swamp of Sorrows",
    1436: "Westfall",
    1437: "Wetlands",
    1438: "Teldrassil",
    1439: "Darkshore",
    1440: "Ashenvale",
    1441: "Thousand Needles",
    1442: "Stonetalon Mountains",
    1443: "Desolace",
    1444: "Feralas",
    1445: "Dustwallow Marsh",
    1446: "Tanaris",
    1447: "Azshara",
    1448: "Felwood",
    1449: "Un'Goro Crater",
    1450: "Moonglade",
    1451: "Silithus",
    1452: "Winterspring"
}

instanceCodeDict = {
    33: "Shadowfang Keep",
    34: "Stormwind Stockade",
    36: "Deadmines",
    43: "Wailing Caverns",
    47: "Razorfen Kraul",
    48: "Blackfathom Deeps",
    70: "Uldaman",
    90: "Gnomeregan",
    109: "Sunken Temple",
    129: "Razorfen Downs",
    189: "Scarlet Monastery",
    209: "Zul'Farrak",
    229: "Blackrock Spire",
    230: "Blackrock Depths",
    289: "Scholomance",
    329: "Stratholme",
    349: "Maraudon",
    389: "Ragefire Chasm",
    429: "Dire Maul",
    469: "Blackwing Lair"
}


In [91]:
### Set of classes by race divided by faction
raceClassDict = {
    1:{1,2,4,5,8,9},
    2:{1,3,4,7,8,9},
    3:{1,2,3,4,5},
    4:{1,3,4,5,11},
    5:{1,4,5,8,9},
    6:{1,3,7,11},
    7:{1,4,8,9},
    8:{1,3,4,5,7,8}
}

hordeRaceSet = {2,5,6,8}
allianceRaceSet = {1,3,4,7}

raceFactionDict = {
    -1: "Overall",
    0: "Alliance",
    1: "Horde"
}

In [73]:
### Functions

# Convert to strings from dict
def final_to_strings(df):
    df['class_id'] = df['class_id'].replace(classCodeDict)
    df['race_id'] = df['race_id'].replace(raceCodeDict)
    df['level_accuracy'] = df['level_accuracy']*100
    df['map_id'] = df['map_id'].replace(zoneCodeDict)
    df['map_accuracy'] = df['map_accuracy']*100
    df['map_mode'] = df['map_mode'].replace(zoneCodeDict)
    df['faction'] = df['faction'].replace(raceFactionDict)
    return df

def major_to_strings(df):
    df['level_accuracy'] = df['level_accuracy']*100
    df['map_id'] = df['map_id'].replace(zoneCodeDict)
    df['map_accuracy'] = df['map_accuracy']*100
    df['map_mode'] = df['map_mode'].replace(zoneCodeDict)
    return df

def select_faction(raceNum):
    if raceNum in allianceRaceSet:
        return 0
    elif raceNum in hordeRaceSet:
        return 1
    else:
        return -1

In [74]:
dfTrain = df.copy().dropna().reset_index(drop=True)

In [75]:
model_svc = SVC()
cv_rkf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=1)

In [76]:
dfTrain

,level,class_id,race_id,map_id
0,1,2,1,1429
1,1,1,1,1436
2,1,3,4,1438
3,1,9,1,1429
4,1,5,4,1437
...,...,...,...,...
45383,60,1,1,1451
45384,60,9,1,1447
45385,60,8,5,1416
45386,60,8,7,1453


In [77]:
maps_predicted = []
X = dfTrain[["race_id","class_id","level"]].copy()
y = dfTrain['map_id'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
model_svc.fit(X_train,y_train)
for raceKey in raceCodeDict:
    for classKey in classCodeDict:
        dfMap = dfTrain[(dfTrain["race_id"] == raceKey) & (dfTrain["class_id"] == classKey)].copy()
        X_pred = dfMap[["race_id","class_id","level"]].copy()
        y_pred = dfMap['map_id'].copy()
        if not y_pred.empty:
            map_score = cross_val_score(model_svc, X_pred, y_pred, cv=cv_rkf, n_jobs=-1)
            map_predict = model_svc.predict(X_pred)
            maps_predicted.append({"race_id":raceKey,"class_id":classKey,"map_id":round(np.mean(map_predict),0).astype(int),"map_accuracy":round(np.mean(map_score),3), "map_mode":stats.mode(dfMap.map_id)[0]})
    

dfMapPred = pd.DataFrame(maps_predicted)
dfMapPred

,race_id,class_id,map_id,map_accuracy,map_mode
0,1,1,1433,0.597,1429
1,1,2,1433,0.635,1429
2,1,4,1433,0.590,1429
3,1,5,1432,0.656,1429
4,1,8,1433,0.622,1436
5,1,9,1432,0.717,1429
6,2,1,1427,0.697,1411
7,2,3,1426,0.768,1411
8,2,4,1424,0.751,1411
9,2,7,1427,0.677,1411


In [78]:
levels_predicted = []
X = dfTrain[["race_id","class_id","map_id"]].copy()
y = dfTrain['level'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
model_svc.fit(X_train,y_train)
for raceKey in raceCodeDict:
    for classKey in classCodeDict:
        dfMap = dfTrain[(dfTrain["race_id"] == raceKey) & (dfTrain["class_id"] == classKey)].copy()
        X_pred = dfMap[["race_id","class_id","map_id"]].copy()
        y_pred = dfMap['level'].copy()
        if not y_pred.empty:
            level_score = cross_val_score(model_svc, X_pred, y_pred, cv=cv_rkf, n_jobs=-1)
            level_predict = model_svc.predict(X_pred)
            levels_predicted.append({"race_id":raceKey,"class_id":classKey,"level":round(np.mean(level_predict),0).astype(int),"level_accuracy":round(np.mean(level_score),3), "level_mode":stats.mode(dfMap.level)[0]})
    

dfLevelPred = pd.DataFrame(levels_predicted)
dfLevelPred

,race_id,class_id,level,level_accuracy,level_mode
0,1,1,6,0.075,13
1,1,2,6,0.080,6
2,1,4,6,0.068,8
3,1,5,6,0.105,11
4,1,8,6,0.081,11
5,1,9,6,0.085,6
6,2,1,6,0.092,10
7,2,3,6,0.066,6
8,2,4,6,0.099,10
9,2,7,6,0.135,10


In [79]:
dfRaClaPred = pd.merge(dfMapPred,dfLevelPred, on=["race_id","class_id"])
dfRaClaPred

,race_id,class_id,map_id,map_accuracy,map_mode,level,level_accuracy,level_mode
0,1,1,1433,0.597,1429,6,0.075,13
1,1,2,1433,0.635,1429,6,0.080,6
2,1,4,1433,0.590,1429,6,0.068,8
3,1,5,1432,0.656,1429,6,0.105,11
4,1,8,1433,0.622,1436,6,0.081,11
5,1,9,1432,0.717,1429,6,0.085,6
6,2,1,1427,0.697,1411,6,0.092,10
7,2,3,1426,0.768,1411,6,0.066,6
8,2,4,1424,0.751,1411,6,0.099,10
9,2,7,1427,0.677,1411,6,0.135,10


In [80]:
### Predict Level Overall

def predict_level_map(dfTest):

    X = dfTest[["race_id","class_id","map_id"]].copy()
    y = dfTest['level'].copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

    model_svc.fit(X_train,y_train)
    levelScore = cross_val_score(model_svc, X_test, y_test, cv=cv_rkf, n_jobs=-1)
    predictLevel = model_svc.predict(X)

    ### Predict Zone
    X = dfTest[["race_id","class_id","level"]].copy()
    y = dfTest['map_id'].copy()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

    model_svc.fit(X_train,y_train)
    mapScore = cross_val_score(model_svc, X_test, y_test, cv=cv_rkf, n_jobs=-1)
    predictMap = model_svc.predict(X)
    return {"level":round(np.mean(predictLevel),0),"level_accuracy":round(np.mean(levelScore),2).astype(int),"level_mode":stats.mode(dfTest.level)[0],"map_id":round(np.mean(predictMap),0).astype(int),"map_accuracy":round(np.mean(mapScore),3), "map_mode":stats.mode(dfTest.map_id)[0]}



In [81]:
dfHorde = dfTrain[dfTrain['race_id'].isin(hordeRaceSet)]
dfAlliance = dfTrain[dfTrain['race_id'].isin(allianceRaceSet)]

setOverallMetrics = predict_level_map(dfTrain)
setAllianceMetrics = predict_level_map(dfAlliance)
setHordeMetrics = predict_level_map(dfHorde)

setOverallMetrics.update({"faction": "Overall"})
setAllianceMetrics.update({"faction": "Alliance"})
setHordeMetrics.update({"faction": "Horde"})



In [82]:
dfOverall = pd.DataFrame(setOverallMetrics, index=[0])
dfAlliance = pd.DataFrame(setAllianceMetrics, index=[0])
dfHorde = pd.DataFrame(setHordeMetrics, index=[0])
dfMajor = pd.concat([dfOverall,dfAlliance,dfHorde]).reset_index(drop=True).reindex(columns=['faction','level','level_accuracy','map_id','map_accuracy','level_mode','map_mode'])
dfMajor.level = dfMajor.level.astype(int)

dfFinal = dfRaClaPred.copy().reset_index(drop=True).reindex(columns=['race_id', 'class_id','level','level_accuracy','map_id','map_accuracy','level_mode','map_mode'])

dfBackup = dfFinal.copy()


In [83]:

dfFinal = dfBackup.copy()

In [84]:
dfMajor

,faction,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode
0,Overall,6,0,1429,0.507,6,1436
1,Alliance,6,0,1434,0.597,8,1436
2,Horde,13,0,1415,0.658,13,1413


In [85]:
dfFinal

,race_id,class_id,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode
0,1,1,6,0.075,1433,0.597,13,1429
1,1,2,6,0.080,1433,0.635,6,1429
2,1,4,6,0.068,1433,0.590,8,1429
3,1,5,6,0.105,1432,0.656,11,1429
4,1,8,6,0.081,1433,0.622,11,1436
5,1,9,6,0.085,1432,0.717,6,1429
6,2,1,6,0.092,1427,0.697,10,1411
7,2,3,6,0.066,1426,0.768,6,1411
8,2,4,6,0.099,1424,0.751,10,1411
9,2,7,6,0.135,1427,0.677,10,1411


In [87]:
dfFinal['faction'] = dfFinal['race_id'].apply(select_faction)
dfFinal

,race_id,class_id,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode,faction
0,1,1,6,0.075,1433,0.597,13,1429,0
1,1,2,6,0.080,1433,0.635,6,1429,0
2,1,4,6,0.068,1433,0.590,8,1429,0
3,1,5,6,0.105,1432,0.656,11,1429,0
4,1,8,6,0.081,1433,0.622,11,1436,0
5,1,9,6,0.085,1432,0.717,6,1429,0
6,2,1,6,0.092,1427,0.697,10,1411,1
7,2,3,6,0.066,1426,0.768,6,1411,1
8,2,4,6,0.099,1424,0.751,10,1411,1
9,2,7,6,0.135,1427,0.677,10,1411,1


In [92]:
dfFinalText = final_to_strings(dfFinal.copy())
dfMajorText = major_to_strings(dfMajor.copy())

In [93]:
dfFinal

,race_id,class_id,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode,faction
0,1,1,6,0.075,1433,0.597,13,1429,0
1,1,2,6,0.080,1433,0.635,6,1429,0
2,1,4,6,0.068,1433,0.590,8,1429,0
3,1,5,6,0.105,1432,0.656,11,1429,0
4,1,8,6,0.081,1433,0.622,11,1436,0
5,1,9,6,0.085,1432,0.717,6,1429,0
6,2,1,6,0.092,1427,0.697,10,1411,1
7,2,3,6,0.066,1426,0.768,6,1411,1
8,2,4,6,0.099,1424,0.751,10,1411,1
9,2,7,6,0.135,1427,0.677,10,1411,1


In [95]:
dfMajor

,faction,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode
0,Overall,6,0,1429,0.507,6,1436
1,Alliance,6,0,1434,0.597,8,1436
2,Horde,13,0,1415,0.658,13,1413


In [94]:
dfFinalText

,race_id,class_id,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode,faction
0,Human,Warrior,6,7.5,Redridge Mountains,59.7,13,Elwynn Forest,Alliance
1,Human,Paladin,6,8.0,Redridge Mountains,63.5,6,Elwynn Forest,Alliance
2,Human,Rogue,6,6.8,Redridge Mountains,59.0,8,Elwynn Forest,Alliance
3,Human,Priest,6,10.5,Loch Modan,65.6,11,Elwynn Forest,Alliance
4,Human,Mage,6,8.1,Redridge Mountains,62.2,11,Westfall,Alliance
5,Human,Warlock,6,8.5,Loch Modan,71.7,6,Elwynn Forest,Alliance
6,Orc,Warrior,6,9.2,Searing Gorge,69.7,10,Durotar,Horde
7,Orc,Hunter,6,6.6,Dun Morogh,76.8,6,Durotar,Horde
8,Orc,Rogue,6,9.9,Hillsbrad Foothills,75.1,10,Durotar,Horde
9,Orc,Shaman,6,13.5,Searing Gorge,67.7,10,Durotar,Horde


In [96]:
dfMajorText

,faction,level,level_accuracy,map_id,map_accuracy,level_mode,map_mode
0,Overall,6,0,Elwynn Forest,50.7,6,Westfall
1,Alliance,6,0,Stranglethorn Vale,59.7,8,Westfall
2,Horde,13,0,Eastern Kingdoms,65.8,13,The Barrens


In [99]:
with open('output.txt', 'w') as file:
    for index, row in dfMajorText.iterrows():
        line = f"{row['faction']} most frequently died at Level {row['level_mode']} in {row['map_mode']}. The Model predict that {row['faction']} will die at Level {row['level']} ({row['level_accuracy']:.2f}%) in {row['map_id']} ({row['map_accuracy']:.2f}%)"
        file.write(line + '\n')

with open('output.txt', 'a') as file:
    for index, row in dfFinalText.iterrows():
        line = f"{row['race_id']} {row['class_id']} most frequently died at Level {row['level_mode']} in {row['map_mode']}. The Model predicts that {row['race_id']} {row['class_id']}'s will die at Level {row['level']} ({row['level_accuracy']:.2f}%) in {row['map_id']} ({row['map_accuracy']:.2f}%)"
        file.write(line + '\n')